In [ ]:
# Install req. packages
! pip install numpy

In [1]:
import numpy as np

In [2]:
## Numpy examples

CHANNELS = 16
LENGTH = 100

ES  = 0     # size of the full event representation
ES += 8     # event type
ES += 128   # pitch
ES += 1     # velocity
ES += 1     # bend
ES += 128   # instrument

#- Create a big tensor with zeros
midi_tensor = np.zeros(shape=(LENGTH, CHANNELS, ES))

#- Create an event vector
event = np.zeros(ES)
event[6] = 1
event[8 + 46] = 1
event[136] = 0.45
# ... etc

#- Add the vector 
midi_tensor[24, 4, :] = event

In [3]:
# Save to a compressed tensor
np.savez_compressed('midi.npz', midi=midi_tensor)

In [4]:
# Load
midi_loaded = np.load('./midi.npz')['midi']
midi_loaded.shape

(100, 16, 266)